In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e12/sample_submission.csv
/kaggle/input/playground-series-s3e12/train.csv
/kaggle/input/playground-series-s3e12/test.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [45]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

from sklearn.model_selection import GridSearchCV
import seaborn as sns

import multiprocessing

try:
    from scikeras.wrappers import KerasClassifier
except ImportError:
    !python -m pip install scikeras
    from scikeras.wrappers import KerasClassifier
    

seed = 69
tf.random.set_seed(69)

In [4]:
train_csv = pd.read_csv("/kaggle/input/playground-series-s3e12/train.csv")
test_csv = pd.read_csv("/kaggle/input/playground-series-s3e12/test.csv")
sample_sub = pd.read_csv("/kaggle/input/playground-series-s3e12/sample_submission.csv")

X, y = train_csv.drop(["target", 'id'], axis = 1), train_csv.target
X

,gravity,ph,osmo,cond,urea,calc
0,1.013,6.19,443,14.8,124,1.45
1,1.025,5.40,703,23.6,394,4.18
2,1.009,6.13,371,24.5,159,9.04
3,1.021,4.91,442,20.8,398,6.63
4,1.021,5.53,874,17.8,385,2.21
...,...,...,...,...,...,...
409,1.011,5.21,527,21.4,75,1.53
410,1.024,5.53,577,19.7,224,0.77
411,1.018,6.28,455,22.2,270,7.68
412,1.008,7.12,325,12.6,75,1.03


In [17]:
def create_model():
    model = keras.Sequential([
    layers.BatchNormalization(input_shape = [6]),
    
    layers.Dense(1024, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(1024, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(1024, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(1, activation = 'sigmoid')
    
])
    
    return model


model = KerasClassifier(
    build_fn = create_model,
    verbose = 0,
    loss= 'binary_crossentropy',
    metrics=["binary_accuracy"]
)

In [49]:
early_stopper = EarlyStopping(
    patience = 10,
    min_delta = 0.0001,
    restore_best_weights = True
)

batchSize = [132, 264 , 512]
optimizers = [ 'RMSProp', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(batch_size = batchSize, optimizer = optimizers)
print(param_grid)

{'batch_size': [132, 264, 512], 'optimizer': ['RMSProp', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']}


In [ ]:
%%timeit
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = 4,cv = 3, error_score="raise",verbose = 3)
grid_result = grid.fit(X, y, callbacks = [early_stopper], epochs = 200)
print(grid_result)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV 3/3] END .batch_size=132, optimizer=RMSProp;, score=0.667 total time= 1.3min
[CV 2/3] END .batch_size=132, optimizer=Adagrad;, score=0.696 total time= 1.0min
[CV 3/3] END batch_size=132, optimizer=Adadelta;, score=0.667 total time= 1.5min
[CV 1/3] END ..batch_size=132, optimizer=Adamax;, score=0.580 total time= 1.5min
[CV 3/3] END ...batch_size=132, optimizer=Nadam;, score=0.616 total time= 1.5min
[CV 3/3] END .batch_size=264, optimizer=Adagrad;, score=0.514 total time= 1.5min
[CV 2/3] END ....batch_size=264, optimizer=Adam;, score=0.551 total time= 1.5min
[CV 2/3] END ...batch_size=264, optimizer=Nadam;, score=0.667 total time= 1.6min
[CV 2/3] END .batch_size=512, optimizer=Adagrad;, score=0.449 total time=  47.8s
[CV 3/3] END batch_size=512, optimizer=Adadelta;, score=0.449 total time=  51.1s


/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will rai

GridSearchCV(cv=3, error_score='raise',
             estimator=KerasClassifier(build_fn=<function create_model at 0x784cf4ae9950>, loss='binary_crossentropy', metrics=['binary_accuracy'], verbose=0),
             n_jobs=4,
             param_grid={'batch_size': [132, 264, 512],
                         'optimizer': ['RMSProp', 'Adagrad', 'Adadelta', 'Adam',
                                       'Adamax', 'Nadam']},
             verbose=3)
Fitting 3 folds for each of 18 candidates, totalling 54 fits
GridSearchCV(cv=3, error_score='raise',
             estimator=KerasClassifier(build_fn=<function create_model at 0x784cf4ae9950>, loss='binary_crossentropy', metrics=['binary_accuracy'], verbose=0),
             n_jobs=4,
             param_grid={'batch_size': [132, 264, 512],
                         'optimizer': ['RMSProp', 'Adagrad', 'Adadelta', 'Adam',
                                       'Adamax', 'Nadam']},
             verbose=3)
Fitting 3 folds for each of 18 candidates, totallin

/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will rai

[CV 2/3] END .batch_size=132, optimizer=RMSProp;, score=0.717 total time= 1.4min
[CV 1/3] END batch_size=132, optimizer=Adadelta;, score=0.609 total time= 1.5min
[CV 3/3] END ....batch_size=132, optimizer=Adam;, score=0.638 total time= 1.1min
[CV 2/3] END ..batch_size=132, optimizer=Adamax;, score=0.681 total time= 1.4min
[CV 1/3] END .batch_size=264, optimizer=RMSProp;, score=0.587 total time=  53.8s
[CV 3/3] END .batch_size=264, optimizer=RMSProp;, score=0.667 total time=  45.1s
[CV 1/3] END batch_size=264, optimizer=Adadelta;, score=0.696 total time=  43.6s
[CV 2/3] END batch_size=264, optimizer=Adadelta;, score=0.775 total time=  44.3s
[CV 3/3] END ....batch_size=264, optimizer=Adam;, score=0.551 total time=  51.1s
[CV 2/3] END ..batch_size=264, optimizer=Adamax;, score=0.710 total time=  49.6s
[CV 3/3] END ...batch_size=264, optimizer=Nadam;, score=0.616 total time=  56.4s
[CV 3/3] END .batch_size=512, optimizer=RMSProp;, score=0.616 total time=  46.6s
[CV 1/3] END batch_size=512,

...

/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will rai

[CV 2/3] END ....batch_size=512, optimizer=Adam;, score=0.464 total time=  44.6s
[CV 3/3] END ..batch_size=512, optimizer=Adamax;, score=0.442 total time=  48.8s
[CV 1/3] END .batch_size=132, optimizer=RMSProp;, score=0.572 total time= 1.4min
[CV 2/3] END batch_size=132, optimizer=Adadelta;, score=0.761 total time= 1.6min
[CV 3/3] END ....batch_size=132, optimizer=Adam;, score=0.616 total time= 1.6min
[CV 1/3] END ...batch_size=132, optimizer=Nadam;, score=0.594 total time= 1.6min
[CV 3/3] END .batch_size=264, optimizer=RMSProp;, score=0.638 total time= 1.5min
[CV 3/3] END batch_size=264, optimizer=Adadelta;, score=0.696 total time= 1.5min
[CV 2/3] END ..batch_size=264, optimizer=Adamax;, score=0.703 total time= 1.5min
[CV 1/3] END .batch_size=512, optimizer=RMSProp;, score=0.500 total time=  42.8s
[CV 1/3] END .batch_size=512, optimizer=Adagrad;, score=0.449 total time=  29.8s
[CV 2/3] END batch_size=512, optimizer=Adadelta;, score=0.442 total time=  48.2s
[CV 1/3] END ....batch_size=

/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"


[CV 1/3] END .batch_size=132, optimizer=RMSProp;, score=0.659 total time= 1.3min
[CV 3/3] END .batch_size=132, optimizer=Adagrad;, score=0.688 total time= 1.4min
[CV 1/3] END ....batch_size=132, optimizer=Adam;, score=0.638 total time= 1.5min
[CV 3/3] END ..batch_size=132, optimizer=Adamax;, score=0.688 total time= 1.0min
[CV 2/3] END ...batch_size=132, optimizer=Nadam;, score=0.703 total time= 1.5min
[CV 1/3] END .batch_size=264, optimizer=Adagrad;, score=0.500 total time= 1.4min
[CV 3/3] END batch_size=264, optimizer=Adadelta;, score=0.652 total time=  56.8s
[CV 1/3] END ..batch_size=264, optimizer=Adamax;, score=0.478 total time=  35.3s
[CV 1/3] END ...batch_size=264, optimizer=Nadam;, score=0.529 total time= 1.1min
[CV 1/3] END .batch_size=512, optimizer=RMSProp;, score=0.435 total time=  33.7s
[CV 1/3] END .batch_size=512, optimizer=Adagrad;, score=0.442 total time=  45.4s
[CV 2/3] END batch_size=512, optimizer=Adadelta;, score=0.442 total time= 1.5min
[CV 2/3] END ..batch_size=51

/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/opt/conda/lib/python3.7/site-packages/scikeras/wrappers.py:302: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will rai

In [ ]:
print(f"Best: {grid_result.best_score_} using parameters: {grid_result.best_params_}\n")

means = grid_result.cv_results_['mean_test_score']
std = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, std, param in zip(means, std, params):
    print(f"{mean}({std}) with {param}")

In [ ]:
grid_result.cv_results_

In [ ]:
history = model.fit(
    X, y,
    batch_size = 264,
    epochs = 100,
    validation_data = [X_val, y_val],
    callbacks = [early_stopper],
)

In [ ]:
history_df = pd.DataFrame(history.history)
pd.DataFrame(history_df)

In [ ]:
sns.lineplot(history_df.loc[:, ['loss', 'val_loss']])

In [ ]:
sns.lineplot(history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']])

In [ ]:
print(f"Min loss : {min(history_df.loss)} ")
print(f"Min val loss : {min(history_df.val_loss)} ")
print(f"Max Accuracy : {max(history_df.binary_accuracy)} ")
print(f"Max val Accuracy : {max(history_df.val_binary_accuracy)} ")

In [ ]:
df = pd.DataFrame(model.predict(test_csv.drop('id', axis = 1)))
df = pd.DataFrame({ 'id': test_csv.id,
    'target': df.loc[:,0]},
            )
print(df)
df.to_csv('submission.csv',index = False)